# Data Collection

### Import Libraries For Data Collection

In [52]:
from tapsdk import TapSDK, TapInputMode
from tapsdk.models import AirGestures
import pandas as pd
import numpy as np
import keyboard
import time
import os 
import glob

tap_instance = []
tap_identifiers = []
global x_array 
global y_array 

modes = ["text", "controller", "controller_text", "raw"]

### Functions

In [38]:
class Functionality:
    current_x = None
    current_y = None
    
    # For connection notification
    def on_connect(self, identifier, name, fw):
        print("Tap has connected")
        print(identifier + " Tap: " + str(name), " FW Version: ", fw)
        if identifier not in tap_identifiers:
            tap_identifiers.append(identifier)
        print("Connected taps:")
        for identifier in tap_identifiers:
            print(identifier)

    # For disconnection notification
    def on_disconnect(self, identifier):
        print("Tap has disconnected")
        if identifier in tap_identifiers:
            tap_identifiers.remove(identifier)
        for identifier in tap_identifiers:
            print(identifier)

    # For mouse data
    def on_mouse_event(self, identifier, dx: int, dy: int, isMouse):
        if isMouse:
            self.current_x = dx 
            self.current_y = dy
            # print(str(dx), str(dy))
        else:
            pass
            # print("Air: ", str(dx), str(dy))

    #For adding columns on top 
    def add_top_column(self, df: pd.DataFrame, top_col: str, inplace=False):
        if not inplace:
            df = df.copy()
        
        df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
        return df

### Data Collection

#### Non-Automated Collection

In [40]:
def main(): 
    count = 0 
    start = input(f"Type 'ST' to start running")
    
    while start == "ST":
        global tap_instance
        tap_instance = TapSDK()
        fun = Functionality()
        tap_instance.run()

        # tap_instance.register_connection_events(on_connect)
        # tap_instance.register_disconnection_events(on_disconnect)
        tap_instance.register_mouse_events(fun.on_mouse_event)
        tap_instance.set_input_mode(TapInputMode(modes[2]))   

        trial = True 
        while trial:
            if(keyboard.is_pressed("q")):
                trial = False
                print("You have chosen to end this trial.")
            else: 
                if str(fun.current_x) != "None":
                    # print(str(fun.current_x) + " "+ str(fun.current_y) + " " + "got it")
                    with open(f"Data\\Random_+1\\random_{count}.txt", "a") as my_file:
                        my_file.write(str(fun.current_x) + " " + str(fun.current_y) +"\n")
                pass

        count += 1
        if  count <= 100:  
            start = input("Type 'ST' to keep running")
        else: 
            start = "Finish"
            print(start)

if __name__ == "__main__":
    main() 

input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this trial.
input mode: Controller and Text Mode
You have chosen to end this

[2023-01-28 02:11:23] TAPLib info # TAP disconnected: Tap_D925210 (BluetoothLE#BluetoothLE48:e7:da:32:4d:b6-e2:5c:59:28:ab:bf) @ C:\Users\Tapper\Documents\GitHub\tap-standalonewin-sdk\TAPWinSDK\TAPManager.cs:dw_removed(171)
[2023-01-28 02:11:23] TAPLib info # TAP disconnected: Tap_D925210 (BluetoothLE#BluetoothLE48:e7:da:32:4d:b6-e2:5c:59:28:ab:bf) @ C:\Users\Tapper\Documents\GitHub\tap-standalonewin-sdk\TAPWinSDK\TAPManager.cs:dw_removed(171)
[2023-01-28 02:11:23] TAPLib info # TAP disconnected: Tap_D925210 (BluetoothLE#BluetoothLE48:e7:da:32:4d:b6-e2:5c:59:28:ab:bf) @ C:\Users\Tapper\Documents\GitHub\tap-standalonewin-sdk\TAPWinSDK\TAPManager.cs:dw_removed(171)
[2023-01-28 02:11:23] TAPLib info # TAP disconnected: Tap_D925210 (BluetoothLE#BluetoothLE48:e7:da:32:4d:b6-e2:5c:59:28:ab:bf) @ C:\Users\Tapper\Documents\GitHub\tap-standalonewin-sdk\TAPWinSDK\TAPManager.cs:dw_removed(171)
[2023-01-28 02:11:23] TAPLib info # TAP disconnected: Tap_D925210 (BluetoothLE#BluetoothLE48:e7:da:32:4d

#### Data Cleaning and Dumping

In [53]:
# Remove repetitive lines from both CW and CCW text files 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Random"
listing = glob.glob(path1 + '\\*.txt')
count = 0 
for file in listing:
    lines_seen = set() # holds lines already seen
    outfile = open(f"Data\\Cleaned_Random\\random_{count}.txt", "a")
    for line in open(file, "r"):
        if line not in lines_seen: # not a duplicate
            outfile.write(line)
            lines_seen.add(line)
    outfile.close()
    count += 1 

In [54]:
#Add data from text files to csv
trial_df = pd.DataFrame()
count = 0 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Cleaned_Random"
listing = glob.glob(path1 + '\\*.txt')
for file in listing:
    x_array = np.array([])
    y_array = np.array([])
    # con = input("Press Y")
    with open(file, 'r') as my_file: 
        for line in my_file:
            line = line.split(" ")
            x_array = np.append((x_array), [int(line[0])])
            y_array = np.append((y_array), [int(line[1])])
    trial_column = pd.DataFrame()
    trial_column[f"x_{count}"] = x_array.tolist()
    trial_column[f"y_{count}"] = y_array.tolist()
    trial_df= pd.concat([trial_df, trial_column],axis=1)
    count += 1 
print("Finished all!!")
trial_df.to_csv(input("Type file name"), index=False)

Finished all!!


#### Experimental Automated Collection

In [ ]:
def main(): 
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            fun = Functionality()
            tap_instance.run()
            tap_instance.register_mouse_events(fun.on_mouse_event)

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1]))   

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else:          
                #append each position change value to the array 
                x_array = np.append(x_array,[fun.current_x])
                y_array = np.append(x_array,[fun.current_y])   
                print(str(fun.current_x), str(fun.current_y)) 
                pass

            # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()

In [ ]:
def main():
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            tap_instance.run()
            tap_instance.register_connection_events(on_connect)
            tap_instance.register_disconnection_events(on_disconnect)
            tap_instance.register_mouse_events(on_mouse_event)
            
            #append each position change value to the array 
            x_array = np.append(x_array,[dx])
            y_array = np.append(x_array,[dy])   
            print(str(dx), str(dy)) 

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1])) 

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else: 
                pass
        # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()

# Model Training and Evaluation

#### Import Libraries

In [19]:
import pandas as pd

#### Relative to Non-Relative

In [69]:
"""
Data given by tapstrap 2 is in the form of relative coordinates, that should be changed to 
non-relative coordinates.
"""
#Dataframe for non-relative coordinates(x,y)
trial_df = pd.DataFrame()

#import dataframe with relative coordinates (x,y)
data = pd.read_csv("Data\\Cleaned_Circle_+1.csv")

for count in range(100):
    #starting point for the drawing of each circle
    origin_x = 0 
    origin_y = 0 
    #array containing calculated coordinates
    new_x_array = np.array([origin_x])
    new_y_array = np.array([origin_y])

    data_temp = data[[f"x_{count}" , f"y_{count}"]]
    data_temp = data_temp.dropna(how='any',axis=0) 
    # data = data.drop_duplicates(keep='first')
    
    for x,y in zip(data_temp[f'x_{count}'].tolist(), data_temp[f'y_{count}'].tolist()): 
        origin_x += x 
        origin_y += y 
        new_x_array = np.append(new_x_array, [origin_x])
        new_y_array = np.append(new_y_array, [origin_y])

    new_df = pd.DataFrame()
    new_df[f"x_{count}"] = new_x_array.tolist()
    new_df[f"y_{count}"] = new_y_array.tolist()
    trial_df = pd.concat([trial_df, new_df], axis= 1)
trial_df.to_csv(input("Type in file name"), index=False)

#### Data Normalization

In [78]:

"""Data was normalized using z-score normalization or standardization.
"""
from sklearn.preprocessing import StandardScaler

data_array = [] # array for non-normalized data
norm_data_array = [] #array for normalized data

#import non-relative coordinate data
data_array.append(pd.read_csv("Data\\Non-Relative_Circle_+1.csv"))
data_array.append(pd.read_csv("Data\\Non-Relative_Circle_-1.csv"))
data_array.append(pd.read_csv("Data\\Non-Relative_Random.csv"))

#iterate through each dataframe in data_array
for data in data_array: 
    data_new = pd.DataFrame()
    for count in range(100):
        #normalize a subset of data
        data_temp = data[[f'x_{count}', f'y_{count}']].dropna(how='any',axis=0) 
        data_temp = data_temp.to_numpy()

        #create standardizing object and normalize aformentioned subset
        scaler = StandardScaler()
        scaler.fit(data_temp)
        data_temp = scaler.transform(data_temp)
        data_temp = pd.DataFrame(data_temp, columns = [f'x_{count}', f'y_{count}'])
        data_new = pd.concat([data_new, data_temp], axis=1)
    
    norm_data_array.append(data_new)

In [123]:
cw = norm_data_array[0]
ccw = norm_data_array[1]
rand_= norm_data_array[2]

In [138]:
# label = np.array([0])
# for x,y in zip(cw[f'x_{0}'].dropna(how='any',axis=0).to_list(), cw[f'y_{0}'].dropna(how='any',axis=0).tolist()):
    
x = cw[f'x_{0}'].dropna(how='any',axis=0).to_numpy()
y = cw[f'y_{0}'].dropna(how='any',axis=0).to_numpy()
out =  np.full(y.shape, 1, dtype=int)
combined_array = np.vstack((x, y, out)).T
temp = pd.DataFrame(combined_array, columns = ['x', 'y','out'])
temp


,x,y,out
0,-1.711644,0.196035,1.0
1,-1.711644,0.191328,1.0
2,-1.711644,0.177208,1.0
3,-1.711644,0.167794,1.0
4,-1.707021,0.153673,1.0
...,...,...,...
254,-1.166059,0.600826,1.0
255,-1.184553,0.572585,1.0
256,-1.184553,0.549051,1.0
257,-1.189177,0.516103,1.0


#### Combine CW and CCW

In [150]:
""" This is to randomize the combination of the CW and CCW data.
"""

import random
import numpy as np

cw = norm_data_array[0]
ccw = norm_data_array[1]
rand_= norm_data_array[2]

mixedFinalData = pd.DataFrame()
cw_set = set()
cww_set= set()
rand_set = set()
count = 0
overall_count = [0, 100, 200]
oc_index = 0 

string_array = ['CW', 'CCW', 'random']


for times in range(300):
    random_string = random.choice(string_array)
    if (random_string == "CW"):
        if count not in cw_set:
            cw_set.add(count)
            prev_count = count 

            x = cw[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = cw[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 1, dtype=int)
            combined_array = np.vstack((x, y, out)).T

            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
            
    elif(random_string == "CCW"):
        if count not in cww_set:
            cww_set.add(count)
            prev_count = count 

            x = ccw[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = ccw[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 1, dtype=int)
            combined_array = np.vstack((x, y, out)).T

            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
    else: 
    
        if count not in rand_set:
            rand_set.add(count)
            prev_count = count 
            x = rand_[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = rand_[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 0, dtype=int)
            combined_array = np.vstack((x, y, out)).T
            
            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
    
    count += 1 
    if(count == 100): 
        oc_index += 1 
        count = 0
    else: 
        pass

mixedFinalData.to_csv(input("Type in a name"), index=False)

In [147]:
""" 
Plot the non-relative (x,y) data just for checking
"""
import matplotlib.pyplot as plt

# data = pd.read_csv("Data\\Non-Relative_Circle_-1.csv")

data = pd.read_csv("Data\\Non-RiskyAllData.csv")
count = 100 # number for trail 
data_temp = data[[f'x_{count}', f'y_{count}']].dropna(how='any',axis=0)  #subset of dataframe

# make data
x,y = data_temp[f'x_{count}'].to_numpy(), data_temp[f'y_{count}'].to_numpy() # from dataframe to arrays containing x and y coordinates

# naming the x and y axis
plt.xlabel('x - axis')
plt.ylabel('y - axis')
  
# plotting a line plot after changing it's width and height
f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)
print("Plot")
plt.plot(x,y)
# plt.show()

KeyError: "None of [Index(['x_100', 'y_100'], dtype='object')] are in the [columns]"

## SVM

## Lazy Predict

### Import Libraries

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier